# Riiid! SAINT+ Solution

This notebook is our solution of the "Riiid! Answer Correctness Prediction".

All codes are available on [GitHub](https://github.com/marisakamozz/riiid).

In [ ]:
!ls /kaggle/input/riiid-saintp-solution/

In [ ]:
import sys
sys.path.append('/kaggle/input/riiid-saintp-solution/')

In [ ]:
import torch
import pandas as pd

from saintmodel import SaintModel, SaintLightningModule, SaintHistory
from saintsubmit import load_saint_config, SaintPredictor, make_submission

In [ ]:
args = load_saint_config()
model = SaintModel(
    seq_len=args.seq_len, n_dim=args.n_dim, std=args.std, dropout=args.dropout, nhead=args.nhead, n_layers=args.n_layers
)
module = SaintLightningModule(args, model)

In [ ]:
%%time
module.load_state_dict(torch.load('/kaggle/input/riiid-saintp-solution/saint.ckpt')['state_dict'])

In [ ]:
%%time
last_history = pd.read_pickle('/kaggle/input/riiid-saintp-solution/last_history.pickle')
last_timestamp = pd.read_pickle('/kaggle/input/riiid-saintp-solution/last_timestamp.pickle')
last_user_count = pd.read_pickle('/kaggle/input/riiid-saintp-solution/last_user_count.pickle')
dict_lag = pd.read_pickle('/kaggle/input/riiid-saintp-solution/dict_lag.pickle')
dict_elapsed = pd.read_pickle('/kaggle/input/riiid-saintp-solution/dict_elapsed.pickle')
dict_user_count = pd.read_pickle('/kaggle/input/riiid-saintp-solution/dict_user_count.pickle')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
module.to(device)

In [ ]:
saint_history = SaintHistory(
    args, last_history, last_timestamp, last_user_count,
    dict_lag, dict_user_count, dict_elapsed
)
predictor = SaintPredictor(module.model, saint_history)

In [ ]:
import riiideducation
env = riiideducation.make_env()

In [ ]:
make_submission(env, predictor)